In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [11]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [12]:
dependent=dataset[['Purchased']]

In [13]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [14]:
independent.shape

(400, 3)

In [15]:
dependent.shape

(400, 1)

In [16]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [18]:
from sklearn.tree import DecisionTreeClassifier

In [19]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],'max_features':['auto','sqrt','log2'],
             'splitter':['best','random']}

grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    2.5s finished
C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
       

In [20]:
re = grid.cv_results_
grid_prediction = grid.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(y_test,grid_prediction)

In [22]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [23]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("the f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)

the f1_macro value for the best parameter {'criterion': 'gini', 'max_features': 'auto', 'splitter': 'best'}: 0.8794289739855382


In [24]:
print ("the confusion Matrix:\n",cf)

the confusion Matrix:
 [[79  6]
 [10 39]]


In [25]:
print ("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [26]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8626650660264106

In [27]:
table=pd.DataFrame.from_dict(re)

In [28]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011371,0.008919,0.001097,0.001552,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",0.885874,0.854934,0.874519,0.871882,0.012789,1
1,0.005555,0.002504,0.001847,0.000239,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",0.792166,0.810298,0.766234,0.789586,0.018016,11
2,0.001848,0.001662,0.003458,0.002447,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.831336,0.831176,0.908359,0.856764,0.036278,2
3,0.005562,0.001148,0.002026,0.000008,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.817398,0.793809,0.896438,0.835743,0.043753,6
4,0.002702,0.000942,0.003144,0.000795,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.843254,0.854397,0.861245,0.852892,0.007434,3
5,0.000000,0.000000,0.004446,0.000516,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.829715,0.772727,0.804383,0.802481,0.023346,9
6,0.003385,0.001264,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",0.852419,0.876195,0.823629,0.850760,0.021412,5
7,0.001156,0.001634,0.001353,0.001914,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",0.817398,0.843509,0.706048,0.789199,0.059432,12
8,0.002326,0.003290,0.000000,0.000000,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.794573,0.842837,0.851704,0.829441,0.025193,7
9,0.001807,0.001610,0.000000,0.000000,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.841852,0.820386,0.770899,0.811277,0.029717,8
